In [1]:
from affine import *
from common import *
from conditions import *
from cycle import *

最もナイーブな方法(ランダムに生成して、条件を満たすものが見つかるまで探索し続ける)

In [10]:
# count = 0
# while(1):
#     count += 1
#     if count % 1000 == 0:
#         print("count: ", count)
#     f_list = [generate_random_affine_function()]*l_h
#     g_list = [generate_random_affine_function()]*l_h
#     if check_conditions(f_list, g_list):
#         print(f_list, g_list)
#         break

先に$f$のリストを作り、条件(a)の解空間に含まれるものをランダムに生成し、条件(b),(c)を確かめる

In [ ]:
cols_list = make_cols_list()

In [ ]:
check_c_results = []
check_c_count = 0
for i in range(10000):
    flag_f = False
    flag_g = False
    not_found = False
    count_f = 0
    count_g = 0
    f_list = []
    while(flag_f is False and not_found is False):
        if count_f == 50:
            not_found = True
        for _ in range(l_h):
            f_list.append(generate_random_affine_function())
        flag_f = check_inner_commute(f_list)
        count_f += 1
    g_candidates = []
    for i in range(l_h):
        g_candidates.append(commute_with(f_list[i]))
    g_candidates_list = list(set(commute_with(f_list[0]))&set(commute_with(f_list[1]))&set(commute_with(f_list[2])))
    while(flag_g is False and not_found is False):
        if count_g == 50:
            not_found = True
        g_list = []
        for _ in range(l_h):
            g_list.append(random.choice(g_candidates_list))
        flag_g = check_inner_commute(g_list)
        count_g += 1
    if not_found:
        continue
    # print(f_list, g_list)
    result = check_condition_c(f_list, g_list, cols_list)
    if result is True:
        print("check success!")
        print(f_list)
        print(g_list)
        break
    else:
        check_c_count += 1
        check_c_results.extend(result)

counts = Counter(check_c_results)
print(check_c_count, len(check_c_results), len(check_c_results)/check_c_count)
print(counts)

57 160284 2812.0
Counter({(0, 1): 57, (2, 4): 57, (1, 2): 57, (0, 4): 57, (3, 4): 57, (1, 5): 57, (0, 3): 57, (1, 4): 57, (2, 3): 57, (0, 2): 57, (4, 5): 57, (0, 5): 57, (2, 5): 57, (1, 3): 57, (3, 5): 57, (0, 1, 4, 5): 57, (0, 4, 0, 4): 57, (0, 1, 5, 4): 57, (1, 3, 5, 2): 57, (0, 5, 2, 4): 57, (0, 5, 4, 1): 57, (1, 3, 2, 3): 57, (1, 5, 2, 5): 57, (1, 5, 4, 2): 57, (0, 4, 1, 5): 57, (0, 4, 2, 4): 57, (1, 4, 2, 3): 57, (0, 3, 1, 3): 57, (0, 5, 1, 5): 57, (0, 1, 2, 1): 57, (0, 5, 3, 2): 57, (1, 3, 1, 4): 57, (1, 3, 5, 4): 57, (0, 3, 2, 4): 57, (1, 2, 1, 3): 57, (2, 5, 3, 5): 57, (1, 3, 2, 5): 57, (1, 4, 2, 5): 57, (0, 3, 1, 5): 57, (2, 3, 5, 3): 57, (0, 1, 2, 3): 57, (0, 5, 3, 4): 57, (0, 1, 3, 2): 57, (1, 5, 3, 2): 57, (1, 2, 1, 5): 57, (1, 2, 3, 2): 57, (0, 3, 5, 2): 57, (0, 2, 1, 2): 57, (0, 2, 4, 1): 57, (1, 2, 5, 2): 57, (0, 1, 2, 5): 57, (0, 1, 4, 2): 57, (0, 1, 3, 4): 57, (0, 1, 5, 1): 57, (0, 2, 3, 2): 57, (1, 5, 3, 4): 57, (3, 4, 3, 4): 57, (0, 5, 2, 1): 57, (0, 2, 0, 3): 57, (1

In [5]:
print(check_c_count, len(check_c_results), len(check_c_results)/check_c_count)

57 160284 2812.0


出力: 57 160284 2812.0